<a href="https://colab.research.google.com/github/nicolashg7/AI/blob/main/AI_Data_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install openai
from google.colab import drive
from transformers import pipeline
import openai

drive.mount('/content/drive')
path = ""
openai.api_key = ""

# Load chatGPT
def ask_chatgpt(prompt):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Load a file
def load_file(path):
  with open(path, 'r') as file:
    text = file.read()
  return text



In [ ]:
# Mount files

files = ["cape_town.txt", "madrid.txt", "rio_de_janeiro.txt", "sydney.txt", "tokyo.txt"]

# Load the model with GPU
pipe = pipeline(task = "text-classification", model = "classla/multilingual-IPTC-news-topic-classifier", device = 0)

for text_file in files:
  # Read the file
  f = open(path + text_file, "r")
  text = f.read()
  f.close()

  # Run the model
  classified = pipe(text)
  relevancy = "relevant" if classified[0]['label'] == 'lifestyle and leisure' else "irrelevant"

  print(f"The text {text_file} is {relevancy}")


In [ ]:
files = ["cape_town.txt"]

# Produce html
for text_file in files:
  # Read the file
  f = path + text_file
  with open(f,'r') as file:
    text = file.read()

  # Run the model
  prompt = f"""
    Produce a html text for the following text: {text}
    Highlight in pink the restaurant names and in green the dishes.
  """
  html = ask_chatgpt(prompt)
  # Write the answer in a new file
  file_path = path + text_file[:-4] + '.html'
  with open(file_path, 'w') as file:
    file.write(html)


In [ ]:
import csv
from IPython.display import display, Markdown

file_path = path + 'itinerary.csv'

with open(file_path, "r") as file:
  itinerary = file.read()

dic_itinerary = []
for row in csv.DictReader(itinerary.splitlines()):
  dic_itinerary.append(row)


prompt = f""" I will visit {dic_itinerary[2]["City"]} in {dic_itinerary[2]["Country"]},
from {dic_itinerary[2]["Arrival"]} to {dic_itinerary[2]["Departure"]}.
Please, provide a detailed itinerary for this visit in Markdown format. Do not include any other text at the end.
"""

response = ask_chatgpt(prompt)
display(Markdown(response))


In [ ]:
import csv

files_names = ["itinerary.csv","cape_town.txt", "rio_de_janeiro.txt", "sydney.txt", "tokyo.txt"]
name_cities = ["Cape Town", "Rio de Janeiro", "Sydney", "Tokyo"]

#Load files
files_content = []
for file in files_names:
    content = load_file(path + file)
    files_content.append(content)

itinerary = []
for row in csv.DictReader(files_content[0].splitlines()):
  itinerary.append(row)

files_content = files_content[1:]

dict_resto = {}
restaurants = {}
for content, name in zip(files_content, name_cities):
  prompt = f"""Extract a list with the name of the restaurants and the most famous dish
  for each of restaurant in the following text: {content}.
  Provide your answer in CSV format, ready to save.
  Exclude the csv declaration, don't add spaces after the comma, do not include column headers.
  Format:
  Res_1, Dish_1
  Res_2, Dish_2
  ...
  """
  response = ask_chatgpt(prompt)

  for row in csv.reader(response.splitlines()):
    restaurant, dish = row  # Unpack each row
    restaurants[restaurant.strip()] = dish.strip()  # Use restaurant as key and dish as value

  name = name.replace(".txt", "")
  dict_resto[name] = restaurants
  restaurants = {}

travel_itinerary = {}
for travel_spot in itinerary:

  prompt = f"""I will travel to {travel_spot['City']} in {travel_spot['Country']},
  from {travel_spot['Arrival']} to {travel_spot['Departure']}.
  Please, provide a detailed itinerary for this visit in Markdown format. Do not include any other text at the end.
  For each day provide a spot for having lunch and for having dinner and choose the restaurant from the following list:
  {dict_resto[travel_spot['City']]}. Do not repeate the same restaurant in any of the days and include
  the dish for each restaurant.
  """
  travel_itinerary[travel_spot['City']] = ask_chatgpt(prompt)


In [ ]:
from IPython.display import display, Markdown
display(Markdown(travel_itinerary['Tokyo']))